In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.ml.regression import LinearRegression

In [11]:
from pyspark.ml.linalg import Vectors

In [12]:
from pyspark.ml.feature import VectorAssembler

In [3]:
spark = SparkSession.builder.appName('reg_ejemplo').getOrCreate()

23/10/24 03:44:31 WARN Utils: Your hostname, ubuntu-s-1vcpu-2gb-sfo3-01 resolves to a loopback address: 127.0.1.1; using 10.48.0.5 instead (on interface eth0)
23/10/24 03:44:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/24 03:44:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
datos = spark.read.csv('customers.csv', inferSchema = True, header = True)

In [7]:
datos.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
datos.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
datos.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [13]:
datos.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 
                                         'Length of Membership', 'Yearly Amount Spent']
                            , outputCol = 'features')

In [19]:
output = assembler.transform(datos)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [22]:
datos_finales = output.select('features', 'Yearly Amount Spent')

In [23]:
datos_finales.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train, test = datos_finales.randomSplit([0.7, 0.3])

In [26]:
train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean| 499.55598934580667|
| stddev|   78.1125856420896|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [27]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                142|
|   mean| 498.70404889786334|
| stddev|  82.54764422796589|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [31]:
lr_modelo = lr.fit(train)

23/10/24 03:54:49 WARN Instrumentation: [6b4b53e0] regParam is zero, which might cause numerical instability and overfitting.


In [32]:
resultado = lr_modelo.evaluate(test)

In [33]:
resultado.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.648459146963432...|
|3.865352482534945...|
|-2.16004991671070...|
|-4.37694325228221...|
|3.808509063674137...|
|5.684341886080801...|
|-4.54747350886464...|
|5.684341886080801...|
|-2.89901436190120...|
|3.922195901395753...|
|-8.52651282912120...|
|-2.21689333557151...|
|-9.09494701772928...|
|7.389644451905042...|
|5.684341886080801...|
|2.785327524179592...|
|-1.13686837721616...|
|-3.52429196937009...|
|-4.77484718430787...|
|2.614797267597168...|
+--------------------+
only showing top 20 rows



In [35]:
resultado.rootMeanSquaredError

2.317901884438273e-12

In [36]:
resultado.r2

1.0